In [4]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import datetime
import csv
import math
import time
from ProgressBar import ProgressBar

import nltk
import string

In [3]:
data = pd.read_csv('2000_Output.csv', header=None)
data.columns = ['id','date','headline', 'lead']
data.head()

,id,date,headline,lead
0,4fd233bf8eb7c8105d7c430b,2000-01-08T00:00:00Z,THE MARKETS: COMMODITIES,CRUDE OIL FALLS. Crude oil declined more than ...
1,4fd237718eb7c8105d7c9aa8,2000-01-08T00:00:00Z,DIMINISHED BRAZILIAN INFLATION,Concern over Brazilian price increases eased a...
2,4fd21c448eb7c8105d79c973,2000-01-08T00:00:00Z,Merrill Lynch Reimburses Client For Loss Linke...,Merrill Lynch & Company said today that it rei...
3,4fd1f22e8eb7c8105d7496d1,2000-01-08T00:00:00Z,NEW UNICOM-PECO MERGER TERMS INCLUDE STOCK BUY...,The Unicom Corporation and the Peco Energy Com...
4,4fd203328eb7c8105d768db0,2000-01-08T00:00:00Z,SYSCO AGREES TO BUY FRUIT AND VEGETABLE DISTRI...,"The Sysco Corporation, North America's largest..."


In [5]:
def getStems(lead):
    stemmer = nltk.stem.SnowballStemmer("english")
    tokens = nltk.word_tokenize(''.join(ch for ch in lead if ch not in set(string.punctuation)))
    return map(lambda x: stemmer.stem(x), tokens)

In [6]:
stems = map(getStems, data.lead)

In [7]:
def getBigrams(stems):
    finder = nltk.collocations.BigramCollocationFinder.from_words(stems)
    return(finder.ngram_fd.items())

In [8]:
bigramList = map(getBigrams, stems)

In [9]:
flatBigrams = [item for sublist in bigramList for item in sublist]

In [10]:
def reduceByKey(toIterate):
    d = {}
    for i, (key, value) in enumerate(toIterate):
        if i%1000==0:
            print i
        if key in d.keys():
            d[key] = d[key]+value
        else:
            d[key] = value
    return list(d.items())

In [11]:
len(flatBigrams)

1386999

In [ ]:
summedBigrams = reduceByKey(flatBigrams)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000


In [ ]:
%store summedBigrams

In [ ]:
def xLogX(x):
    if x==0:
        return 0
    else:
        return x * np.log(x)
    
def entropy(things):
    theSum = 0
    theResult = 0
    for thing in things:
        theResult += xLogX(thing)
        theSum += thing;

    return xLogX(theSum) - theResult;

def logLikelihoodRatio(k11, k12, k21, k22):
    rowEntropy = entropy([k11 + k12, k21 + k22]);
    columnEntropy = entropy([k11 + k21, k12 + k22]);
    matrixEntropy = entropy([k11, k12, k21, k22]);
    return(2.0 * (rowEntropy + columnEntropy - matrixEntropy))

def getLLR(summedBigram):
    k11 = summedBigram[1]
    stem1 = summedBigram[0][0]
    stem2 = summedBigram[0][1]
    k12 = bigrams1Dict[stem1] - k11
    k21 = bigrams2Dict[stem2] - k11
    k22 = totalBigrams - k11 - k12 - k21
    return(logLikelihoodRatio(k11, k12, k21, k22))

In [ ]:
bigrams1 = map(lambda x: (x[0][0], x[1]), summedBigrams)
summedBigrams1= reduceByKey(bigrams1)

In [ ]:
bigrams2 = map(lambda x: (x[0][1], x[1]), summedBigrams)
summedBigrams2= reduceByKey(bigrams2)

In [ ]:
bigrams1Dict = dict(summedBigrams1)
bigrams2Dict = dict(summedBigrams2)

In [ ]:
onlyValues = np.array(map(lambda x: x[1], summedBigrams))
totalBigrams = onlyValues.sum()

In [ ]:
LLRs = map(getLLR, summedBigrams)

In [ ]:
plt.hist(LLRs, bins=5000);
plt.xlim((0,50))
plt.xlabel('Log-likelihood Ratio')
plt.ylabel('Counts')
plt.title('Distribution of Bigram Log-Likelihood Ratios')

In [ ]:
bigramVocab = np.array(summedBigrams)[list(np.where(np.array(LLRs)>25)[0])]